# AlgoSpace Synergy Strategy 1: MLMI → FVG → NW-RQK

## Strategy Overview
This notebook implements the first synergy pattern, using Machine Learning Market Intelligence (MLMI), Fair Value Gaps (FVG), and Nadaraya-Watson with Rational Quadratic Kernel (NW-RQK) in a sequential confirmation pattern.

### Key Components:
1. **MLMI**: Advanced KNN-based pattern recognition for momentum analysis
2. **FVG**: Detects price inefficiencies and gaps for entry zones
3. **NW-RQK**: Kernel regression for trend direction confirmation

### Signal Generation:
- Long Entry: MLMI bullish → FVG bullish zone → NW-RQK bullish confirmation
- Short Entry: MLMI bearish → FVG bearish zone → NW-RQK bearish confirmation

In [ ]:
# Cell 1: Complete Environment Setup, Imports, and Configuration
# This cell contains ALL necessary imports, configurations, and setup
# No pre-initialization of variables - they will be created where needed

import pandas as pd
import numpy as np
import vectorbt as vbt
import numba
from numba import jit, njit, prange, typed, types, float64, int64, boolean
from numba.typed import Dict
from numba.experimental import jitclass
from scipy.spatial import cKDTree
import warnings
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dataclasses import dataclass
import json
import os
from typing import Tuple, Dict, Any, Optional

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Configure Numba for optimal performance
numba.config.THREADING_LAYER = 'threadsafe'
numba.config.NUMBA_NUM_THREADS = numba.config.NUMBA_DEFAULT_NUM_THREADS

# Performance settings
pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:.4f}'.format

# Configure VectorBT settings
vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 1200
vbt.settings['plotting']['layout']['height'] = 600

# Complete Strategy Configuration
@dataclass
class StrategyConfig:
    """Comprehensive configuration for the Synergy 1 strategy"""
    # Data paths
    data_5m_path: str = "/home/QuantNova/AlgoSpace-8/notebooks/notebook data/@CL - 5 min - ETH.csv"
    data_30m_path: str = "/home/QuantNova/AlgoSpace-8/notebooks/notebook data/@CL - 30 min - ETH.csv"
    
    # MLMI parameters
    mlmi_ma_fast_period: int = 5
    mlmi_ma_slow_period: int = 20
    mlmi_rsi_fast_period: int = 5
    mlmi_rsi_slow_period: int = 20
    mlmi_rsi_smooth_period: int = 20
    mlmi_k_neighbors: int = 200
    mlmi_max_data_size: int = 10000
    
    # FVG parameters
    fvg_lookback: int = 10
    fvg_body_multiplier: float = 1.5
    fvg_validity: int = 20
    
    # NW-RQK parameters
    nwrqk_h: float = 8.0
    nwrqk_r: float = 8.0
    nwrqk_x_0: int = 25
    nwrqk_lag: int = 2
    nwrqk_smooth_colors: bool = False
    
    # Synergy parameters
    synergy_window: int = 30
    mlmi_threshold: float = 0.0
    
    # Backtesting parameters
    initial_capital: float = 100000.0
    position_size: float = 100.0
    fees: float = 0.0001
    slippage: float = 0.0001
    max_hold_bars: int = 100
    stop_loss: float = 0.01
    take_profit: float = 0.05
    
    # Data validation
    min_data_points: int = 100

# Initialize configuration
config = StrategyConfig()
print("✅ Environment setup complete")
print(f"✅ Numba version: {numba.__version__}")
print(f"✅ VectorBT version: {vbt.__version__}")
print(f"✅ Configuration loaded with {len(config.__dataclass_fields__)} parameters")

In [ ]:
# Cell 2: Robust Data Loading with Column Standardization
# This cell loads and validates data with comprehensive column name standardization

def standardize_column_names(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize column names to handle various naming conventions"""
    # Create a mapping of common variations to standard names
    column_mapping = {
        # Timestamp variations
        'timestamp': 'Timestamp', 'datetime': 'Timestamp', 'date': 'Timestamp',
        'time': 'Timestamp', 'Time': 'Timestamp', 'Date': 'Timestamp',
        'DateTime': 'Timestamp', 'TIMESTAMP': 'Timestamp',
        
        # Open variations
        'open': 'Open', 'o': 'Open', 'OPEN': 'Open', 'O': 'Open',
        
        # High variations
        'high': 'High', 'h': 'High', 'HIGH': 'High', 'H': 'High',
        
        # Low variations
        'low': 'Low', 'l': 'Low', 'LOW': 'Low', 'L': 'Low',
        
        # Close variations
        'close': 'Close', 'c': 'Close', 'CLOSE': 'Close', 'C': 'Close',
        
        # Volume variations
        'volume': 'Volume', 'v': 'Volume', 'VOLUME': 'Volume', 'V': 'Volume',
        'vol': 'Volume', 'Vol': 'Volume', 'VOL': 'Volume'
    }
    
    # Apply mapping
    df = df.rename(columns=column_mapping)
    
    return df

def validate_dataframe(df: pd.DataFrame, name: str, min_data_points: int = 100) -> bool:
    """Validate dataframe has required columns and data quality"""
    required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
    
    # Check for required columns
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        print(f"⚠️ Warning: {name} missing columns: {missing_cols}")
        # Continue with available columns
    
    # Check for sufficient data
    if len(df) < min_data_points:
        raise ValueError(f"{name} has insufficient data: {len(df)} rows (minimum: {min_data_points})")
    
    # Check for valid price data
    price_cols = ['Open', 'High', 'Low', 'Close']
    for col in price_cols:
        if col in df.columns:
            if df[col].isna().all():
                raise ValueError(f"{name} column '{col}' contains only NaN values")
            if (df[col] <= 0).any():
                print(f"⚠️ Warning: {name} column '{col}' contains non-positive values")
    
    return True

def load_data_optimized(file_path: str, timeframe: str = '5m') -> pd.DataFrame:
    """Load and prepare data with comprehensive error handling and column standardization"""
    start_time = time.time()
    
    try:
        # Check if file exists
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")
        
        # Read CSV with intelligent parsing
        df = pd.read_csv(file_path, low_memory=False)
        
        # Standardize column names
        df = standardize_column_names(df)
        
        # Handle timestamp parsing
        timestamp_col = None
        for col in ['Timestamp', 'timestamp', 'DateTime', 'datetime', 'Date', 'date', 'Time', 'time']:
            if col in df.columns:
                timestamp_col = col
                break
        
        if timestamp_col:
            # Try multiple date formats
            for date_format in [None, '%Y-%m-%d %H:%M:%S', '%d/%m/%Y %H:%M:%S', '%m/%d/%Y %H:%M:%S']:
                try:
                    if date_format:
                        df['Timestamp'] = pd.to_datetime(df[timestamp_col], format=date_format)
                    else:
                        df['Timestamp'] = pd.to_datetime(df[timestamp_col], infer_datetime_format=True)
                    break
                except:
                    continue
            
            # Set timestamp as index
            df.set_index('Timestamp', inplace=True)
        else:
            print(f"⚠️ Warning: No timestamp column found in {timeframe} data")
        
        # Ensure numeric types for fast operations
        numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').astype(np.float64)
        
        # Remove any NaN values in critical columns
        critical_cols = ['Open', 'High', 'Low', 'Close']
        existing_critical = [col for col in critical_cols if col in df.columns]
        
        if existing_critical:
            initial_len = len(df)
            df.dropna(subset=existing_critical, inplace=True)
            dropped = initial_len - len(df)
            if dropped > 0:
                print(f"📊 Dropped {dropped} rows with NaN values")
        
        # Validate OHLC relationships
        if all(col in df.columns for col in ['Open', 'High', 'Low', 'Close']):
            invalid_ohlc = ((df['High'] < df['Low']) | 
                           (df['High'] < df['Open']) | 
                           (df['High'] < df['Close']) | 
                           (df['Low'] > df['Open']) | 
                           (df['Low'] > df['Close']))
            if invalid_ohlc.any():
                print(f"⚠️ Warning: Found {invalid_ohlc.sum()} rows with invalid OHLC relationships")
                df = df[~invalid_ohlc]
        
        # Sort index for faster operations
        df.sort_index(inplace=True)
        
        # Check for duplicate timestamps
        if df.index.duplicated().any():
            print(f"⚠️ Warning: Found {df.index.duplicated().sum()} duplicate timestamps, keeping first")
            df = df[~df.index.duplicated(keep='first')]
        
        load_time = time.time() - start_time
        print(f"✅ Loaded {len(df):,} rows in {load_time:.2f} seconds from {timeframe} file")
        
        # Validate the loaded data
        validate_dataframe(df, f"{timeframe} data", config.min_data_points)
        
        return df
        
    except Exception as e:
        print(f"❌ Error loading {file_path}: {str(e)}")
        raise

# Load the data files
print("Loading data files...")
print(f"5m data path: {config.data_5m_path}")
print(f"30m data path: {config.data_30m_path}")

try:
    # Load 5-minute data
    print("\n📊 Loading 5-minute data...")
    df_5m = load_data_optimized(config.data_5m_path, '5m')
    
    # Load 30-minute data
    print("\n📊 Loading 30-minute data...")
    df_30m = load_data_optimized(config.data_30m_path, '30m')
    
    # Verify time alignment
    print("\n⏰ Verifying time alignment...")
    
    # Find overlapping period
    start_time = max(df_5m.index[0], df_30m.index[0])
    end_time = min(df_5m.index[-1], df_30m.index[-1])
    
    if start_time >= end_time:
        raise ValueError("No overlapping time period between 5m and 30m data")
    
    # Trim dataframes to overlapping period
    df_5m = df_5m[start_time:end_time]
    df_30m = df_30m[start_time:end_time]
    
    print(f"\n✅ Aligned data period: {start_time} to {end_time}")
    print(f"📊 5-minute bars after alignment: {len(df_5m):,}")
    print(f"📊 30-minute bars after alignment: {len(df_30m):,}")
    
    # Verify reasonable ratio
    ratio = len(df_5m) / len(df_30m)
    expected_ratio = 6  # 30min / 5min
    if abs(ratio - expected_ratio) > 1:
        print(f"⚠️ Warning: Unexpected timeframe ratio: {ratio:.2f} (expected ~{expected_ratio})")
    else:
        print(f"✅ Timeframe ratio verified: {ratio:.2f}")
    
    print(f"\n📅 5-minute data: {df_5m.index[0]} to {df_5m.index[-1]}")
    print(f"📅 30-minute data: {df_30m.index[0]} to {df_30m.index[-1]}")
    
    print("\n✅ Data loading completed successfully!")
    
except Exception as e:
    print(f"\n❌ Fatal error during data loading: {str(e)}")
    print("Cannot proceed with analysis. Please check your data files.")
    raise

In [ ]:
# Cell 3: Original Indicator Implementations
# This cell contains ALL the original, correct indicator functions from Strategy_Implementation.ipynb

# ============================================================================
# MLMI IMPLEMENTATION SECTION
# ============================================================================

# Define spec for jitclass
spec = [
    ('parameter1', float64[:]),
    ('parameter2', float64[:]),
    ('priceArray', float64[:]),
    ('resultArray', int64[:]),
    ('size', int64)
]

# Create a JIT-compiled MLMI data class for maximum performance
@jitclass(spec)
class MLMIDataFast:
    def __init__(self, max_size=10000):
        # Pre-allocate arrays with maximum size for better performance
        self.parameter1 = np.zeros(max_size, dtype=np.float64)
        self.parameter2 = np.zeros(max_size, dtype=np.float64)
        self.priceArray = np.zeros(max_size, dtype=np.float64)
        self.resultArray = np.zeros(max_size, dtype=np.int64)
        self.size = 0
    
    def storePreviousTrade(self, p1, p2, close_price):
        if self.size > 0:
            # Calculate result before modifying current values
            result = 1 if close_price >= self.priceArray[self.size-1] else -1
            
            # Increment size and add new entry
            self.size += 1
            self.parameter1[self.size-1] = p1
            self.parameter2[self.size-1] = p2
            self.priceArray[self.size-1] = close_price
            self.resultArray[self.size-1] = result
        else:
            # First entry
            self.parameter1[0] = p1
            self.parameter2[0] = p2
            self.priceArray[0] = close_price
            self.resultArray[0] = 0  # Neutral for first entry
            self.size = 1

@njit(fastmath=True, parallel=True)
def wma_numba_fast(series, length):
    """Ultra-optimized Weighted Moving Average calculation"""
    n = len(series)
    result = np.zeros(n, dtype=np.float64)
    
    # Pre-calculate weights (constant throughout calculation)
    weights = np.arange(1, length + 1, dtype=np.float64)
    sum_weights = np.sum(weights)
    
    # Parallel processing of WMA calculation
    for i in prange(length-1, n):
        weighted_sum = 0.0
        # Inline loop for better performance
        for j in range(length):
            weighted_sum += series[i-j] * weights[length-j-1]
        result[i] = weighted_sum / sum_weights
    
    return result

@njit(fastmath=True)
def calculate_rsi_numba_fast(prices, window):
    """Ultra-optimized RSI calculation"""
    n = len(prices)
    rsi = np.zeros(n, dtype=np.float64)
    
    # Pre-allocate arrays for better memory performance
    delta = np.zeros(n, dtype=np.float64)
    gain = np.zeros(n, dtype=np.float64)
    loss = np.zeros(n, dtype=np.float64)
    avg_gain = np.zeros(n, dtype=np.float64)
    avg_loss = np.zeros(n, dtype=np.float64)
    
    # Calculate deltas in one pass
    for i in range(1, n):
        delta[i] = prices[i] - prices[i-1]
        # Separate gains and losses in the same loop
        if delta[i] > 0:
            gain[i] = delta[i]
        else:
            loss[i] = -delta[i]
    
    # First value uses simple average
    if window <= n:
        avg_gain[window-1] = np.sum(gain[:window]) / window
        avg_loss[window-1] = np.sum(loss[:window]) / window
        
        # Calculate RSI for first window point
        if avg_loss[window-1] == 0:
            rsi[window-1] = 100
        else:
            rs = avg_gain[window-1] / avg_loss[window-1]
            rsi[window-1] = 100 - (100 / (1 + rs))
    
    # Apply Wilder's smoothing for subsequent values with optimized calculation
    window_minus_one = window - 1
    window_recip = 1.0 / window
    for i in range(window, n):
        avg_gain[i] = (avg_gain[i-1] * window_minus_one + gain[i]) * window_recip
        avg_loss[i] = (avg_loss[i-1] * window_minus_one + loss[i]) * window_recip
        
        # Calculate RSI directly
        if avg_loss[i] == 0:
            rsi[i] = 100
        else:
            rs = avg_gain[i] / avg_loss[i]
            rsi[i] = 100 - (100 / (1 + rs))
    
    return rsi

# Use cKDTree for lightning-fast kNN queries
def fast_knn_predict(param1_array, param2_array, result_array, p1, p2, k, size):
    """
    Ultra-fast kNN prediction using scipy.spatial.cKDTree
    """
    # Handle empty data case
    if size == 0:
        return 0
    
    # Create points array for KDTree
    points = np.column_stack((param1_array[:size], param2_array[:size]))
    
    # Create KDTree for fast nearest neighbor search
    tree = cKDTree(points)
    
    # Query KDTree for k nearest neighbors
    distances, indices = tree.query([p1, p2], k=min(k, size))
    
    # Get results of nearest neighbors
    neighbors = result_array[indices]
    
    # Return prediction (sum of neighbor results)
    return np.sum(neighbors)

def calculate_mlmi_optimized(df, num_neighbors=200, momentum_window=20):
    """
    Highly optimized MLMI calculation function
    """
    print("Preparing data for MLMI calculation...")
    # Get numpy arrays for better performance
    close_array = df['Close'].values
    n = len(close_array)
    
    # Pre-allocate all output arrays at once
    ma_quick = np.zeros(n, dtype=np.float64)
    ma_slow = np.zeros(n, dtype=np.float64)
    rsi_quick = np.zeros(n, dtype=np.float64)
    rsi_slow = np.zeros(n, dtype=np.float64)
    rsi_quick_wma = np.zeros(n, dtype=np.float64)
    rsi_slow_wma = np.zeros(n, dtype=np.float64)
    pos = np.zeros(n, dtype=np.bool_)
    neg = np.zeros(n, dtype=np.bool_)
    mlmi_values = np.zeros(n, dtype=np.float64)
    
    print("Calculating RSI and moving averages...")
    # Calculate indicators with optimized functions
    ma_quick = wma_numba_fast(close_array, 5)
    ma_slow = wma_numba_fast(close_array, 20)
    
    # Calculate RSI with optimized function
    rsi_quick = calculate_rsi_numba_fast(close_array, 5)
    rsi_slow = calculate_rsi_numba_fast(close_array, 20)
    
    # Apply WMA to RSI values
    rsi_quick_wma = wma_numba_fast(rsi_quick, momentum_window)
    rsi_slow_wma = wma_numba_fast(rsi_slow, momentum_window)
    
    # Detect MA crossovers (vectorized where possible)
    print("Detecting moving average crossovers...")
    for i in range(1, n):
        if ma_quick[i] > ma_slow[i] and ma_quick[i-1] <= ma_slow[i-1]:
            pos[i] = True
        if ma_quick[i] < ma_slow[i] and ma_quick[i-1] >= ma_slow[i-1]:
            neg[i] = True
    
    # Initialize optimized MLMI data object
    mlmi_data = MLMIDataFast(max_size=min(10000, n))  # Pre-allocate with reasonable size
    
    print("Processing crossovers and calculating MLMI values...")
    # Process data with batch processing for performance
    crossover_indices = np.where(pos | neg)[0]
    
    # Process crossovers in a single pass
    for i in crossover_indices:
        if not np.isnan(rsi_slow_wma[i]) and not np.isnan(rsi_quick_wma[i]):
            mlmi_data.storePreviousTrade(
                rsi_slow_wma[i],
                rsi_quick_wma[i],
                close_array[i]
            )
    
    # Batch kNN predictions for performance
    # Only calculate for points after momentum_window
    for i in range(momentum_window, n):
        if not np.isnan(rsi_slow_wma[i]) and not np.isnan(rsi_quick_wma[i]):
            # Use fast KDTree-based kNN prediction
            if mlmi_data.size > 0:
                mlmi_values[i] = fast_knn_predict(
                    mlmi_data.parameter1,
                    mlmi_data.parameter2,
                    mlmi_data.resultArray,
                    rsi_slow_wma[i],
                    rsi_quick_wma[i],
                    num_neighbors,
                    mlmi_data.size
                )
    
    # Add results to dataframe (do this all at once)
    df_result = df.copy()
    df_result['ma_quick'] = ma_quick
    df_result['ma_slow'] = ma_slow
    df_result['rsi_quick'] = rsi_quick
    df_result['rsi_slow'] = rsi_slow
    df_result['rsi_quick_wma'] = rsi_quick_wma
    df_result['rsi_slow_wma'] = rsi_slow_wma
    df_result['pos'] = pos
    df_result['neg'] = neg
    df_result['mlmi'] = mlmi_values
    
    # Calculate WMA of MLMI
    df_result['mlmi_ma'] = wma_numba_fast(mlmi_values, 20)
    
    # Calculate bands and other derived values
    print("Calculating bands and crossovers...")
    
    # Use vectorized operations for bands calculation
    highest_values = pd.Series(mlmi_values).rolling(window=2000, min_periods=1).max().values
    lowest_values = pd.Series(mlmi_values).rolling(window=2000, min_periods=1).min().values
    mlmi_std = pd.Series(mlmi_values).rolling(window=20).std().values
    ema_std = pd.Series(mlmi_std).ewm(span=20).mean().values
    
    # Add band values to dataframe
    df_result['upper'] = highest_values
    df_result['lower'] = lowest_values
    df_result['upper_band'] = highest_values - ema_std
    df_result['lower_band'] = lowest_values + ema_std
    
    # Generate crossover signals (vectorized where possible)
    mlmi_bull_cross = np.zeros(n, dtype=np.bool_)
    mlmi_bear_cross = np.zeros(n, dtype=np.bool_)
    mlmi_ob_cross = np.zeros(n, dtype=np.bool_)
    mlmi_ob_exit = np.zeros(n, dtype=np.bool_)
    mlmi_os_cross = np.zeros(n, dtype=np.bool_)
    mlmi_os_exit = np.zeros(n, dtype=np.bool_)
    mlmi_mid_up = np.zeros(n, dtype=np.bool_)
    mlmi_mid_down = np.zeros(n, dtype=np.bool_)
    
    # Calculate crossovers in one pass for better performance
    for i in range(1, n):
        if not np.isnan(mlmi_values[i]) and not np.isnan(mlmi_values[i-1]):
            # MA crossovers
            if mlmi_values[i] > df_result['mlmi_ma'].iloc[i] and mlmi_values[i-1] <= df_result['mlmi_ma'].iloc[i-1]:
                mlmi_bull_cross[i] = True
            if mlmi_values[i] < df_result['mlmi_ma'].iloc[i] and mlmi_values[i-1] >= df_result['mlmi_ma'].iloc[i-1]:
                mlmi_bear_cross[i] = True
                
            # Overbought/Oversold crossovers
            if mlmi_values[i] > df_result['upper_band'].iloc[i] and mlmi_values[i-1] <= df_result['upper_band'].iloc[i-1]:
                mlmi_ob_cross[i] = True
            if mlmi_values[i] < df_result['upper_band'].iloc[i] and mlmi_values[i-1] >= df_result['upper_band'].iloc[i-1]:
                mlmi_ob_exit[i] = True
            if mlmi_values[i] < df_result['lower_band'].iloc[i] and mlmi_values[i-1] >= df_result['lower_band'].iloc[i-1]:
                mlmi_os_cross[i] = True
            if mlmi_values[i] > df_result['lower_band'].iloc[i] and mlmi_values[i-1] <= df_result['lower_band'].iloc[i-1]:
                mlmi_os_exit[i] = True
                
            # Zero-line crosses
            if mlmi_values[i] > 0 and mlmi_values[i-1] <= 0:
                mlmi_mid_up[i] = True
            if mlmi_values[i] < 0 and mlmi_values[i-1] >= 0:
                mlmi_mid_down[i] = True
    
    # Add crossover signals to dataframe
    df_result['mlmi_bull_cross'] = mlmi_bull_cross
    df_result['mlmi_bear_cross'] = mlmi_bear_cross
    df_result['mlmi_ob_cross'] = mlmi_ob_cross
    df_result['mlmi_ob_exit'] = mlmi_ob_exit
    df_result['mlmi_os_cross'] = mlmi_os_cross
    df_result['mlmi_os_exit'] = mlmi_os_exit
    df_result['mlmi_mid_up'] = mlmi_mid_up
    df_result['mlmi_mid_down'] = mlmi_mid_down
    
    # Count signals
    bull_crosses = np.sum(mlmi_bull_cross)
    bear_crosses = np.sum(mlmi_bear_cross)
    ob_cross = np.sum(mlmi_ob_cross)
    ob_exit = np.sum(mlmi_ob_exit)
    os_cross = np.sum(mlmi_os_cross)
    os_exit = np.sum(mlmi_os_exit)
    zero_up = np.sum(mlmi_mid_up)
    zero_down = np.sum(mlmi_mid_down)
    
    print(f"\nMLMI Signal Summary:")
    print(f"- Bullish MA Crosses: {bull_crosses}")
    print(f"- Bearish MA Crosses: {bear_crosses}")
    print(f"- Overbought Crosses: {ob_cross}")
    print(f"- Overbought Exits: {ob_exit}")
    print(f"- Oversold Crosses: {os_cross}")
    print(f"- Oversold Exits: {os_exit}")
    print(f"- Zero Line Crosses Up: {zero_up}")
    print(f"- Zero Line Crosses Down: {zero_down}")
    
    return df_result

# ============================================================================
# FVG IMPLEMENTATION SECTION
# ============================================================================

def detect_fvg(df, lookback_period=10, body_multiplier=1.5):
    """
    Detects Fair Value Gaps (FVGs) in historical price data.
    
    Parameters:
        df (DataFrame): DataFrame with OHLC data
        lookback_period (int): Number of candles to look back for average body size
        body_multiplier (float): Multiplier to determine significant body size
        
    Returns:
        list: List of FVG tuples or None values
    """
    # Create a list to store FVG results
    fvg_list = [None] * len(df)
    
    # Can't form FVG with fewer than 3 candles
    if len(df) < 3:
        print("Warning: Not enough data points to detect FVGs")
        return fvg_list
    
    # Start from the third candle (index 2)
    for i in range(2, len(df)):
        try:
            # Get the prices for three consecutive candles
            first_high = df['High'].iloc[i-2]
            first_low = df['Low'].iloc[i-2]
            middle_open = df['Open'].iloc[i-1]
            middle_close = df['Close'].iloc[i-1]
            third_low = df['Low'].iloc[i]
            third_high = df['High'].iloc[i]
            
            # Calculate average body size from lookback period
            start_idx = max(0, i-1-lookback_period)
            prev_bodies = (df['Close'].iloc[start_idx:i-1] - df['Open'].iloc[start_idx:i-1]).abs()
            avg_body_size = prev_bodies.mean() if not prev_bodies.empty else 0.001
            avg_body_size = max(avg_body_size, 0.001)  # Avoid division by zero
            
            # Calculate current middle candle body size
            middle_body = abs(middle_close - middle_open)
            
            # Check for Bullish FVG (gap up)
            if third_low > first_high and middle_body > avg_body_size * body_multiplier:
                fvg_list[i] = ('bullish', first_high, third_low, i)
                
            # Check for Bearish FVG (gap down)
            elif third_high < first_low and middle_body > avg_body_size * body_multiplier:
                fvg_list[i] = ('bearish', first_low, third_high, i)
                
        except Exception as e:
            # Skip this candle if there's an error
            continue
    
    return fvg_list

def process_fvg_active_zones(df, fvg_list, validity_bars=20):
    """
    Process FVG list to create active zone boolean arrays.
    Properly tracks individual FVGs and their invalidation.
    """
    n = len(df)
    is_bull_fvg_active = np.zeros(n, dtype=bool)
    is_bear_fvg_active = np.zeros(n, dtype=bool)
    
    # Track active FVGs with proper structure
    active_bull_fvgs = []  # List of tuples: (lower_level, upper_level, start_idx)
    active_bear_fvgs = []  # List of tuples: (upper_level, lower_level, start_idx)
    
    for i in range(n):
        # Add new FVGs to active list
        if fvg_list[i] is not None:
            fvg_type, level1, level2, idx = fvg_list[i]
            if fvg_type == 'bullish':
                # For bullish FVG: level1 is first candle high, level2 is third candle low
                active_bull_fvgs.append((level1, level2, idx))
            else:
                # For bearish FVG: level1 is first candle low, level2 is third candle high
                active_bear_fvgs.append((level1, level2, idx))
        
        # Check active bullish FVGs
        new_active_bull = []
        for lower_level, upper_level, start_idx in active_bull_fvgs:
            # FVG remains valid if:
            # 1. Within validity period
            # 2. Price hasn't closed below the lower level (invalidation)
            if i - start_idx < validity_bars and df['Close'].iloc[i] > lower_level:
                is_bull_fvg_active[i] = True
                new_active_bull.append((lower_level, upper_level, start_idx))
        active_bull_fvgs = new_active_bull
        
        # Check active bearish FVGs
        new_active_bear = []
        for upper_level, lower_level, start_idx in active_bear_fvgs:
            # FVG remains valid if:
            # 1. Within validity period
            # 2. Price hasn't closed above the upper level (invalidation)
            if i - start_idx < validity_bars and df['Close'].iloc[i] < upper_level:
                is_bear_fvg_active[i] = True
                new_active_bear.append((upper_level, lower_level, start_idx))
        active_bear_fvgs = new_active_bear
    
    return is_bull_fvg_active, is_bear_fvg_active

# ============================================================================
# NW-RQK IMPLEMENTATION SECTION
# ============================================================================

@njit(float64(float64[:], int64, float64, float64, int64))
def kernel_regression_numba(src, size, h_param, r_param, x_0):
    """
    Numba-optimized Nadaraya-Watson Regression using Rational Quadratic Kernel
    """
    current_weight = 0.0
    cumulative_weight = 0.0
    
    # Calculate only up to the available data points
    for i in range(min(size + x_0 + 1, len(src))):
        if i < len(src):
            y = src[i]  # Value i bars back
            # Rational Quadratic Kernel
            w = (1 + (i**2 / ((h_param**2) * 2 * r_param)))**(-r_param)
            current_weight += y * w
            cumulative_weight += w
    
    if cumulative_weight == 0:
        return np.nan
    
    return current_weight / cumulative_weight

@njit(parallel=True)
def calculate_nw_regression(prices, h_param, h_lag_param, r_param, x_0_param):
    """
    Calculate Nadaraya-Watson regression for the entire price series
    """
    n = len(prices)
    yhat1 = np.full(n, np.nan)
    yhat2 = np.full(n, np.nan)
    
    # Reverse the array once to match PineScript indexing
    prices_reversed = np.zeros(n)
    for i in range(n):
        prices_reversed[i] = prices[n-i-1]
    
    # Calculate regression values for each bar in parallel
    for i in prange(n):
        if i >= x_0_param:  # Only start calculation after x_0 bars
            # Create window for current bar
            window_size = min(i + 1, n)
            src = np.zeros(window_size)
            for j in range(window_size):
                src[j] = prices[i-j]
            
            yhat1[i] = kernel_regression_numba(src, i, h_param, r_param, x_0_param)
            yhat2[i] = kernel_regression_numba(src, i, h_lag_param, r_param, x_0_param)
    
    return yhat1, yhat2

@njit
def detect_crosses(yhat1, yhat2):
    """
    Detect crossovers between two series
    """
    n = len(yhat1)
    bullish_cross = np.zeros(n, dtype=np.bool_)
    bearish_cross = np.zeros(n, dtype=np.bool_)
    
    for i in range(1, n):
        if not np.isnan(yhat1[i]) and not np.isnan(yhat2[i]) and \
           not np.isnan(yhat1[i-1]) and not np.isnan(yhat2[i-1]):
            # Bullish cross (yhat2 crosses above yhat1)
            if yhat2[i] > yhat1[i] and yhat2[i-1] <= yhat1[i-1]:
                bullish_cross[i] = True
            
            # Bearish cross (yhat2 crosses below yhat1)
            if yhat2[i] < yhat1[i] and yhat2[i-1] >= yhat1[i-1]:
                bearish_cross[i] = True
    
    return bullish_cross, bearish_cross

def calculate_nw_rqk(df, src_col='Close', h=8.0, r=8.0, x_0=25, lag=2, smooth_colors=False):
    """
    Calculate Nadaraya-Watson RQK indicator for a dataframe
    """
    print("Calculating Nadaraya-Watson Regression with Rational Quadratic Kernel...")
    
    # Convert to numpy array for Numba
    prices = df[src_col].values
    
    # Calculate regression values using Numba
    yhat1, yhat2 = calculate_nw_regression(prices, h, h-lag, r, x_0)
    
    # Add regression values to dataframe
    df['yhat1'] = yhat1
    df['yhat2'] = yhat2
    
    # Calculate rates of change (vectorized)
    df['wasBearish'] = df['yhat1'].shift(2) > df['yhat1'].shift(1)
    df['wasBullish'] = df['yhat1'].shift(2) < df['yhat1'].shift(1)
    df['isBearish'] = df['yhat1'].shift(1) > df['yhat1']
    df['isBullish'] = df['yhat1'].shift(1) < df['yhat1']
    df['isBearishChange'] = df['isBearish'] & df['wasBullish']
    df['isBullishChange'] = df['isBullish'] & df['wasBearish']
    
    # Calculate crossovers using Numba
    bullish_cross, bearish_cross = detect_crosses(yhat1, yhat2)
    df['isBullishCross'] = bullish_cross
    df['isBearishCross'] = bearish_cross
    
    # Calculate smooth color conditions (vectorized)
    df['isBullishSmooth'] = df['yhat2'] > df['yhat1']
    df['isBearishSmooth'] = df['yhat2'] < df['yhat1']
    
    # Define colors (matches PineScript)
    c_bullish = '#3AFF17'  # Green
    c_bearish = '#FD1707'  # Red
    
    # Determine plot colors based on settings (vectorized)
    df['colorByCross'] = np.where(df['isBullishSmooth'], c_bullish, c_bearish)
    df['colorByRate'] = np.where(df['isBullish'], c_bullish, c_bearish)
    df['plotColor'] = df['colorByCross'] if smooth_colors else df['colorByRate']
    
    # Calculate alert conditions (vectorized)
    df['alertBullish'] = df['isBearishCross'] if smooth_colors else df['isBearishChange']
    df['alertBearish'] = df['isBullishCross'] if smooth_colors else df['isBullishChange']
    
    # Generate alert stream (-1 for bearish, 1 for bullish, 0 for no change) (vectorized)
    df['alertStream'] = np.where(df['alertBearish'], -1,
                                np.where(df['alertBullish'], 1, 0))
    
    # Count signals
    bullish_changes = df['isBullishChange'].sum()
    bearish_changes = df['isBearishChange'].sum()
    bullish_crosses = df['isBullishCross'].sum()
    bearish_crosses = df['isBearishCross'].sum()
    
    print(f"\nNW-RQK Signal Summary:")
    print(f"- Bullish Rate Changes: {bullish_changes}")
    print(f"- Bearish Rate Changes: {bearish_changes}")
    print(f"- Bullish Crosses: {bullish_crosses}")
    print(f"- Bearish Crosses: {bearish_crosses}")
    
    return df

print("✅ All original indicator implementations loaded successfully")

In [ ]:
# Cell 4: Strategy Execution
# This cell performs all calculations in the correct sequence

print("=" * 80)
print("EXECUTING SYNERGY STRATEGY CALCULATIONS")
print("=" * 80)

# Step 1: Calculate MLMI on 30m data
print("\n📊 Step 1: Calculating MLMI on 30-minute data...")
df_30m = calculate_mlmi_optimized(
    df_30m.copy(),
    num_neighbors=config.mlmi_k_neighbors,
    momentum_window=config.mlmi_rsi_smooth_period
)

# CRITICAL FIX: Use MLMI threshold crossings, not MA crossovers
# The MLMI value itself crossing zero is the signal, not the MA crossovers
df_30m['mlmi_bull'] = (df_30m['mlmi'] > config.mlmi_threshold) & (df_30m['mlmi'].shift(1) <= config.mlmi_threshold)
df_30m['mlmi_bear'] = (df_30m['mlmi'] < config.mlmi_threshold) & (df_30m['mlmi'].shift(1) >= config.mlmi_threshold)

print(f"✅ MLMI calculation complete")
print(f"   - Valid MLMI values: {(~df_30m['mlmi'].isna()).sum():,}")
print(f"   - MLMI zero-crossing bull signals: {df_30m['mlmi_bull'].sum():,}")
print(f"   - MLMI zero-crossing bear signals: {df_30m['mlmi_bear'].sum():,}")

# Step 2: Calculate NW-RQK on 30m data
print("\n📊 Step 2: Calculating NW-RQK on 30-minute data...")
df_30m = calculate_nw_rqk(
    df_30m,
    src_col='Close',
    h=config.nwrqk_h,
    r=config.nwrqk_r,
    x_0=config.nwrqk_x_0,
    lag=config.nwrqk_lag,
    smooth_colors=config.nwrqk_smooth_colors
)

print(f"✅ NW-RQK calculation complete")
print(f"   - Valid yhat1 values: {(~df_30m['yhat1'].isna()).sum():,}")
print(f"   - Bullish changes: {df_30m['isBullishChange'].sum():,}")
print(f"   - Bearish changes: {df_30m['isBearishChange'].sum():,}")

# Step 3: Calculate FVG on 5m data
print("\n📊 Step 3: Detecting FVG on 5-minute data...")
fvg_list = detect_fvg(
    df_5m,
    lookback_period=config.fvg_lookback,
    body_multiplier=config.fvg_body_multiplier
)

# Process FVG active zones
is_bull_fvg_active, is_bear_fvg_active = process_fvg_active_zones(
    df_5m,
    fvg_list,
    validity_bars=config.fvg_validity
)

# Add FVG signals to 5m dataframe
df_5m['fvg_bull'] = is_bull_fvg_active
df_5m['fvg_bear'] = is_bear_fvg_active

# Count FVG detections
fvg_bull_count = sum(1 for fvg in fvg_list if fvg is not None and fvg[0] == 'bullish')
fvg_bear_count = sum(1 for fvg in fvg_list if fvg is not None and fvg[0] == 'bearish')

print(f"✅ FVG detection complete")
print(f"   - Bullish FVGs detected: {fvg_bull_count:,}")
print(f"   - Bearish FVGs detected: {fvg_bear_count:,}")
print(f"   - Active bullish zones: {is_bull_fvg_active.sum():,}")
print(f"   - Active bearish zones: {is_bear_fvg_active.sum():,}")

# Step 4: Align 30m indicators to 5m timeframe
print("\n⏰ Step 4: Aligning 30m indicators to 5m timeframe...")

# Create the strategy dataframe starting with 5m data
df_strategy = df_5m.copy()

# SIMPLIFIED ALIGNMENT: Use pandas reindex with forward fill
# This ensures each 5m bar has the corresponding 30m indicator values
indicators_to_align = [
    'mlmi', 'mlmi_bull', 'mlmi_bear',  # MLMI indicators
    'isBullishChange', 'isBearishChange',  # NW-RQK change signals (correct ones)
    'yhat1', 'yhat2',  # NW-RQK regression values
    'mlmi_ma', 'upper_band', 'lower_band'  # Additional MLMI bands for analysis
]

for indicator in indicators_to_align:
    if indicator in df_30m.columns:
        # Simple pandas reindex - much cleaner than complex timestamp mapping
        aligned = df_30m[indicator].reindex(df_strategy.index, method='ffill')
        df_strategy[f'{indicator}_30m'] = aligned

print(f"✅ Timeframe alignment complete")
print(f"   - Strategy dataframe shape: {df_strategy.shape}")
print(f"   - Aligned indicators: {len(indicators_to_align)}")

# Step 5: Generate Synergy Signals (MLMI → FVG → NW-RQK)
print("\n🎯 Step 5: Generating synergy signals with simplified logic...")

# Initialize signal arrays
n = len(df_strategy)
long_entry = np.zeros(n, dtype=bool)
short_entry = np.zeros(n, dtype=bool)

# SIMPLIFIED SYNERGY DETECTION: Window-based approach
# Look back for MLMI signal, check current FVG zone, confirm with NW-RQK change
for i in range(config.synergy_window, n):
    # Look back within the synergy window for MLMI zero-crossing signals
    mlmi_bull_window = df_strategy['mlmi_bull_30m'].iloc[i-config.synergy_window:i+1]
    mlmi_bear_window = df_strategy['mlmi_bear_30m'].iloc[i-config.synergy_window:i+1]
    
    # Check if we had MLMI signal in the lookback window
    had_mlmi_bull = mlmi_bull_window.any()
    had_mlmi_bear = mlmi_bear_window.any()
    
    # Check current FVG zone status (must be in active zone)
    in_bull_fvg = df_strategy['fvg_bull'].iloc[i]
    in_bear_fvg = df_strategy['fvg_bear'].iloc[i]
    
    # Check NW-RQK change confirmation (using the correct change signals)
    nwrqk_bull_confirm = df_strategy['isBullishChange_30m'].iloc[i]
    nwrqk_bear_confirm = df_strategy['isBearishChange_30m'].iloc[i]
    
    # Simple AND logic for synergy signals
    if had_mlmi_bull and in_bull_fvg and nwrqk_bull_confirm:
        long_entry[i] = True
    
    if had_mlmi_bear and in_bear_fvg and nwrqk_bear_confirm:
        short_entry[i] = True

# Add signals to dataframe
df_strategy['long_entry'] = long_entry
df_strategy['short_entry'] = short_entry

# Count total signals
total_long_signals = long_entry.sum()
total_short_signals = short_entry.sum()

print(f"✅ Synergy signal generation complete")
print(f"   - Long entry signals: {total_long_signals:,}")
print(f"   - Short entry signals: {total_short_signals:,}")
print(f"   - Total signals: {total_long_signals + total_short_signals:,}")

# Display sample of signals if any exist
if total_long_signals > 0 or total_short_signals > 0:
    print("\n📋 Sample of generated signals:")
    signal_mask = df_strategy['long_entry'] | df_strategy['short_entry']
    sample_signals = df_strategy[signal_mask].head(5)
    print(sample_signals[['Close', 'long_entry', 'short_entry', 'mlmi_30m', 'fvg_bull', 'fvg_bear']])

print("\n" + "=" * 80)
print("✅ STRATEGY EXECUTION COMPLETE")
print("=" * 80)

In [ ]:
# Cell 5: Backtesting with VectorBT
# This cell runs the backtest on the generated signals

print("=" * 80)
print("BACKTESTING SYNERGY STRATEGY")
print("=" * 80)

# Prepare data for backtesting
print("\n📊 Preparing data for backtest...")

# Extract price and signal data
close_prices = df_strategy['Close'].values
entries = df_strategy[['long_entry', 'short_entry']].values

# Create position sizing arrays
size = np.full_like(entries, config.position_size, dtype=np.float64)
size_type = 'amount'

print(f"✅ Backtest data prepared")
print(f"   - Price data points: {len(close_prices):,}")
print(f"   - Entry signals: {entries.sum():,}")

# Run the backtest
print("\n🚀 Running backtest...")

try:
    portfolio = vbt.Portfolio.from_signals(
        close=close_prices,
        entries=entries,
        short_entries=df_strategy['short_entry'].values,
        size=size,
        size_type=size_type,
        init_cash=config.initial_capital,
        fees=config.fees,
        slippage=config.slippage,
        freq='5T'
    )
    
    print("✅ Backtest completed successfully")
    
    # Calculate and display statistics
    stats = portfolio.stats()
    
    print("\n📈 BACKTEST RESULTS")
    print("-" * 50)
    print(f"Initial Capital: ${config.initial_capital:,.2f}")
    print(f"Final Portfolio Value: ${stats['Total Return [$]'] + config.initial_capital:,.2f}")
    print(f"Total Return: {stats['Total Return [%]']:.2f}%")
    print(f"\nTrade Statistics:")
    print(f"- Total Trades: {stats['Total Trades']:.0f}")
    print(f"- Win Rate: {stats['Win Rate [%]']:.2f}%")
    print(f"- Average Win: {stats['Avg Winning Trade [%]']:.2f}%")
    print(f"- Average Loss: {stats['Avg Losing Trade [%]']:.2f}%")
    print(f"\nRisk Metrics:")
    print(f"- Max Drawdown: {stats['Max Drawdown [%]']:.2f}%")
    print(f"- Sharpe Ratio: {stats['Sharpe Ratio']:.2f}")
    print(f"- Sortino Ratio: {stats['Sortino Ratio']:.2f}")
    print(f"- Calmar Ratio: {stats['Calmar Ratio']:.2f}")
    
    # Store key metrics
    backtest_metrics = {
        'total_return_pct': stats['Total Return [%]'],
        'total_trades': stats['Total Trades'],
        'win_rate': stats['Win Rate [%]'],
        'sharpe_ratio': stats['Sharpe Ratio'],
        'max_drawdown': stats['Max Drawdown [%]']
    }
    
except Exception as e:
    print(f"❌ Error during backtesting: {str(e)}")
    portfolio = None
    stats = None

print("\n" + "=" * 80)

In [ ]:
# Cell 6: Visualization Dashboard
# This cell creates comprehensive visualizations of the strategy

if portfolio is not None:
    print("Creating visualization dashboard...")
    
    # Create subplots
    fig = make_subplots(
        rows=5, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        row_heights=[0.3, 0.2, 0.2, 0.2, 0.1],
        subplot_titles=(
            'Price Action with Signals',
            'MLMI Indicator',
            'NW-RQK Regression',
            'Portfolio Value',
            'FVG Active Zones'
        )
    )
    
    # Get last 1000 bars for visualization
    viz_start = max(0, len(df_strategy) - 1000)
    df_viz = df_strategy.iloc[viz_start:]
    
    # Plot 1: Price with entry signals
    fig.add_trace(
        go.Candlestick(
            x=df_viz.index,
            open=df_viz['Open'],
            high=df_viz['High'],
            low=df_viz['Low'],
            close=df_viz['Close'],
            name='Price',
            showlegend=False
        ),
        row=1, col=1
    )
    
    # Add long entry signals
    long_signals = df_viz[df_viz['long_entry']]
    if len(long_signals) > 0:
        fig.add_trace(
            go.Scatter(
                x=long_signals.index,
                y=long_signals['Low'] * 0.995,
                mode='markers',
                marker=dict(symbol='triangle-up', size=10, color='green'),
                name='Long Entry'
            ),
            row=1, col=1
        )
    
    # Add short entry signals
    short_signals = df_viz[df_viz['short_entry']]
    if len(short_signals) > 0:
        fig.add_trace(
            go.Scatter(
                x=short_signals.index,
                y=short_signals['High'] * 1.005,
                mode='markers',
                marker=dict(symbol='triangle-down', size=10, color='red'),
                name='Short Entry'
            ),
            row=1, col=1
        )
    
    # Plot 2: MLMI Indicator
    fig.add_trace(
        go.Scatter(
            x=df_viz.index,
            y=df_viz['mlmi_30m'],
            mode='lines',
            name='MLMI',
            line=dict(color='blue', width=2)
        ),
        row=2, col=1
    )
    
    # Add zero line
    fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)
    
    # Plot 3: NW-RQK
    fig.add_trace(
        go.Scatter(
            x=df_viz.index,
            y=df_viz['yhat1_30m'],
            mode='lines',
            name='NW-RQK',
            line=dict(color='purple', width=2)
        ),
        row=3, col=1
    )
    
    # Plot 4: Portfolio Value
    portfolio_value = portfolio.value()
    portfolio_value_viz = portfolio_value.iloc[viz_start:]
    
    fig.add_trace(
        go.Scatter(
            x=portfolio_value_viz.index,
            y=portfolio_value_viz.values,
            mode='lines',
            name='Portfolio Value',
            line=dict(color='orange', width=2),
            fill='tozeroy'
        ),
        row=4, col=1
    )
    
    # Plot 5: FVG Active Zones
    fig.add_trace(
        go.Scatter(
            x=df_viz.index,
            y=df_viz['fvg_bull'].astype(int),
            mode='lines',
            name='Bull FVG',
            line=dict(color='green', width=1),
            fill='tozeroy',
            opacity=0.3
        ),
        row=5, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_viz.index,
            y=-df_viz['fvg_bear'].astype(int),
            mode='lines',
            name='Bear FVG',
            line=dict(color='red', width=1),
            fill='tozeroy',
            opacity=0.3
        ),
        row=5, col=1
    )
    
    # Update layout
    fig.update_layout(
        title='Synergy Strategy 1: MLMI → FVG → NW-RQK Performance Dashboard',
        xaxis_title='Date',
        height=1400,
        showlegend=True,
        template='plotly_dark'
    )
    
    # Update y-axis labels
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="MLMI", row=2, col=1)
    fig.update_yaxes(title_text="NW-RQK", row=3, col=1)
    fig.update_yaxes(title_text="Value ($)", row=4, col=1)
    fig.update_yaxes(title_text="Active", row=5, col=1)
    
    # Show the plot
    fig.show()
    
    print("\n✅ Visualization complete!")
else:
    print("⚠️ No portfolio data available for visualization")

In [ ]:
# Cell 7: Export Results and Summary
# This cell exports results and provides a final summary

print("=" * 80)
print("STRATEGY EXECUTION SUMMARY")
print("=" * 80)

# Create results summary
results_summary = {
    "strategy": "Synergy 1: MLMI → FVG → NW-RQK",
    "execution_time": time.strftime("%Y-%m-%d %H:%M:%S"),
    "data_period": {
        "start": str(df_strategy.index[0]),
        "end": str(df_strategy.index[-1]),
        "total_bars_5m": len(df_strategy),
        "total_bars_30m": len(df_30m)
    },
    "signals": {
        "mlmi_bull_crosses": int(df_30m['mlmi_bull'].sum()),
        "mlmi_bear_crosses": int(df_30m['mlmi_bear'].sum()),
        "fvg_bull_zones": int(df_strategy['fvg_bull'].sum()),
        "fvg_bear_zones": int(df_strategy['fvg_bear'].sum()),
        "nwrqk_bull_changes": int(df_30m['isBullishChange'].sum()),
        "nwrqk_bear_changes": int(df_30m['isBearishChange'].sum()),
        "synergy_long_entries": int(df_strategy['long_entry'].sum()),
        "synergy_short_entries": int(df_strategy['short_entry'].sum())
    },
    "backtest_results": backtest_metrics if 'backtest_metrics' in locals() else None,
    "configuration": {
        "mlmi_neighbors": config.mlmi_k_neighbors,
        "fvg_validity_bars": config.fvg_validity,
        "nwrqk_h": config.nwrqk_h,
        "synergy_window": config.synergy_window
    }
}

# Export results to JSON
results_file = 'synergy_1_results.json'
with open(results_file, 'w') as f:
    json.dump(results_summary, f, indent=2)

print(f"✅ Results exported to {results_file}")

# Display final summary
print("\n📊 FINAL PERFORMANCE SUMMARY")
print("-" * 50)

if portfolio is not None and stats is not None:
    print(f"Total Return: {stats['Total Return [%]']:.2f}%")
    print(f"Sharpe Ratio: {stats['Sharpe Ratio']:.2f}")
    print(f"Win Rate: {stats['Win Rate [%]']:.2f}%")
    print(f"Max Drawdown: {stats['Max Drawdown [%]']:.2f}%")
    print(f"Total Trades: {stats['Total Trades']:.0f}")
    
    # Performance rating
    if stats['Sharpe Ratio'] > 1.5:
        rating = "⭐⭐⭐⭐⭐ Excellent"
    elif stats['Sharpe Ratio'] > 1.0:
        rating = "⭐⭐⭐⭐ Very Good"
    elif stats['Sharpe Ratio'] > 0.5:
        rating = "⭐⭐⭐ Good"
    elif stats['Sharpe Ratio'] > 0:
        rating = "⭐⭐ Fair"
    else:
        rating = "⭐ Needs Improvement"
    
    print(f"\nPerformance Rating: {rating}")

print("\n🎯 KEY INSIGHTS:")
print(f"1. The strategy generated {results_summary['signals']['synergy_long_entries'] + results_summary['signals']['synergy_short_entries']} total signals")
print(f"2. Signal ratio (Long/Short): {results_summary['signals']['synergy_long_entries']}/{results_summary['signals']['synergy_short_entries']}")

if portfolio is not None:
    print(f"3. Average trade duration: {portfolio.stats()['Avg Trade Duration']:.1f}")
    print(f"4. Profit factor: {portfolio.stats()['Profit Factor']:.2f}")

print("\n" + "=" * 80)
print("✅ SYNERGY STRATEGY 1 EXECUTION COMPLETE!")
print("=" * 80)